In [179]:
%pip install langchain 
%pip install -U langchain-community
%pip install pymysql

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [180]:
from langchain_groq import ChatGroq
import os
os.environ["GROQ_API_KEY"] = "yourAPIKey"
llm = ChatGroq(
    model="gemma2-9b-it",
    temperature=0.5,
    groq_api_key=os.environ["GROQ_API_KEY"]
)
response = llm.invoke("What is the capital of France?")
print(response.content)


The capital of France is **Paris**. 🗼  



In [181]:
from langchain.utilities import SQLDatabase

db = SQLDatabase.from_uri(
    f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",
    sample_rows_in_table_info=3
)
print(db.table_info)



CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount

*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Adidas	Black	S	48

In [182]:
%pip install langchain-experimental


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [183]:
from langchain_experimental.sql import SQLDatabaseChain

In [184]:
db_chain = SQLDatabaseChain.from_llm(
    llm, db,
    verbose=True
)
qns1=db_chain.run("How many tshirts do we have left for nike in extra small size and white colour?")




> Entering new SQLDatabaseChain chain...
How many tshirts do we have left for nike in extra small size and white colour?
SQLQuery:SQLQuery: 
SELECT 
  SUM(`stock_quantity`) 
FROM 
  `t_shirts` 
WHERE 
  `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White';
SQLResult: [(Decimal('78'),)]
Answer:Question: How many tshirts do we have left for nike in extra small size and white colour?
SQLQuery:SELECT 
  SUM(`stock_quantity`) 
FROM 
  `t_shirts` 
WHERE 
  `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White';
> Finished chain.


In [185]:
qns1

"Question: How many tshirts do we have left for nike in extra small size and white colour?\nSQLQuery:SELECT \n  SUM(`stock_quantity`) \nFROM \n  `t_shirts` \nWHERE \n  `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White';"

In [186]:
qns2=db_chain.run("What is the price of inventory for all small size tshirts?")



> Entering new SQLDatabaseChain chain...
What is the price of inventory for all small size tshirts?
SQLQuery:Question: What is the price of inventory for all small size tshirts?
SQLQuery: SELECT SUM(`price`) FROM `t_shirts` WHERE `size` = 'S';
SQLResult: [(Decimal('397'),)]
Answer:Question: What is the price of inventory for all small size tshirts?
SQLQuery:SELECT SUM(`price`) FROM `t_shirts` WHERE `size` = 'S';
> Finished chain.


In [187]:
qns2=db_chain.run("SELECT SUM(price*stock_quantity) FROM `t_shirts` WHERE `size` = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM `t_shirts` WHERE `size` = 'S'
SQLQuery:Question: SELECT SUM(price*stock_quantity) FROM `t_shirts` WHERE `size` = 'S'
SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `size` = 'S';
SQLResult: [(Decimal('25644'),)]
Answer:Question: SELECT SUM(price*stock_quantity) FROM `t_shirts` WHERE `size` = 'S'
SQLQuery:SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `size` = 'S';
> Finished chain.


In [188]:
qns3=db_chain.run("If we have to sell all Levi's t-shirts today with discounts applied. How much revenue our store will generate(post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all Levi's t-shirts today with discounts applied. How much revenue our store will generate(post discounts)?
SQLQuery:Question: If we have to sell all Levi's t-shirts today with discounts applied. How much revenue our store will generate(post discounts)?
SQLQuery: 
SELECT SUM(`price` * (1 - d.`pct_discount`/100)) 
FROM `t_shirts` AS ts
JOIN `discounts` AS d ON ts.`t_shirt_id` = d.`t_shirt_id`
WHERE ts.`brand` = 'Levi' AND CURDATE() = DATE(d.`discount_id`)
SQLResult: [(None,)]
Answer:Question: If we have to sell all Levi's t-shirts today with discounts applied. How much revenue our store will generate(post discounts)?
SQLQuery:SELECT SUM(`price` * (1 - d.`pct_discount`/100)) 
FROM `t_shirts` AS ts
JOIN `discounts` AS d ON ts.`t_shirt_id` = d.`t_shirt_id`
WHERE ts.`brand` = 'Levi' AND CURDATE() = DATE(d.`discount_id`)
> Finished chain.


In [189]:
qns4=db_chain.run("SELECT SUM(price*stock_quantity) FROM `t_shirts` WHERE `brand` = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM `t_shirts` WHERE `brand` = 'Levi'
SQLQuery:Question: SELECT SUM(price*stock_quantity) FROM `t_shirts` WHERE `brand` = 'Levi'
SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi';
SQLResult: [(Decimal('20186'),)]
Answer:Question: SELECT SUM(price*stock_quantity) FROM `t_shirts` WHERE `brand` = 'Levi'
SQLQuery:SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi';
> Finished chain.


In [190]:
qns4=db_chain.run("SELECT SUM(price*stock_quantity) FROM `t_shirts` WHERE `brand` = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM `t_shirts` WHERE `brand` = 'Levi'
SQLQuery:Question: SELECT SUM(price*stock_quantity) FROM `t_shirts` WHERE `brand` = 'Levi'
SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi';
SQLResult: [(Decimal('20186'),)]
Answer:Question: SELECT SUM(price*stock_quantity) FROM `t_shirts` WHERE `brand` = 'Levi'
SQLQuery:SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi';
> Finished chain.


In [191]:
qns5 = db_chain.run("How many white color Levi's t-shirts do we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t-shirts do we have available?
SQLQuery:Question: How many white color Levi's t-shirts do we have available?
SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White';
SQLResult: [(74,), (41,), (93,), (63,), (12,)]
Answer:Question: How many white color Levi's t-shirts do we have available?
SQLQuery:SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White';
> Finished chain.


In [192]:
qns5=db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' and color='white'")




> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' and color='white'
SQLQuery:Question: SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' and color='white'
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'white';
SQLResult: [(Decimal('283'),)]
Answer:Question: SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' and color='white'
SQLQuery:SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'white';
> Finished chain.


In [193]:
few_shots = [
    {
        'Question': "How many t-shirts do we have left for Nike in extra small size and white color?",
        'SQLQuery': "SELECT * FROM t_shirts WHERE brand = 'Nike' and size='XS' and color='white'",
        'SQLResult': "Result of the SQL query",
        'Answer': qns1
    },
    {
        'Question': "What is the price of inventory for all small size t-shirts?",
        'SQLQuery': "SELECT SUM(price*stock_quantity) FROM `t_shirts` WHERE `size` = 'S'",
        'SQLResult': "Result of the SQL query",
        'Answer': qns2
    },
    {
        'Question': "If we have to sell all Levi's t-shirts today. How much revenue our store will generate?",
        'SQLQuery': "SELECT SUM(price*stock_quantity) FROM `t_shirts` WHERE `brand` = 'Levi'",
        'SQLResult': "Result of the SQL query",
        'Answer': qns3
    },
    {
        'Question': "How many white color Levi's t-shirts do we have available?",
        'SQLQuery': "SELECT * FROM t_shirts WHERE brand = 'Levi' and color='white'",
        'SQLResult': "Result of the SQL query",
        'Answer': qns4
    }
]

In [194]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2")
e=embeddings.embed_query("How many white color Levi's t-shirts do I have?")


In [195]:
e[:5]

[0.00989247765392065,
 0.03737413138151169,
 0.002890500472858548,
 -0.00859028473496437,
 0.035603396594524384]

In [196]:
to_vectorize = [ " ".join(example.values()) for example in few_shots ]

In [197]:
from langchain.vectorstores import Chroma

# The error is because metadatas should be a list of dicts (one per text), not a string.
vectorestore = Chroma.from_texts(to_vectorize,
    embedding=embeddings,
    metadatas=few_shots
)

In [198]:
from langchain.prompts import SemanticSimilarityExampleSelector
example_selector = SemanticSimilarityExampleSelector(vectorstore=vectorestore, k=2)

example_selector.select_examples({"Question": "How many Adidas tshirts I have left in my store?"})


[{'source': 'few_shots'},
 {'SQLResult': 'Result of the SQL query',
  'Question': 'How many t-shirts do we have left for Nike in extra small size and white color?',
  'Answer': "Question: How many tshirts do we have left for nike in extra small size and white colour?\nSQLQuery:SELECT  `stock_quantity` FROM  `t_shirts` WHERE  `brand` = 'Nike' AND  `size` = 'XS' AND  `color` = 'White';",
  'SQLQuery': "SELECT * FROM t_shirts WHERE brand = 'Nike' and size='XS' and color='white'"}]

In [199]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [200]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [201]:
from langchain.prompts.prompt import PromptTemplate
example_prompt= PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult", "Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}\n\n"
)


In [202]:
from langchain.prompts import FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"],
)

In [203]:
new_chain = SQLDatabaseChain.from_llm(
    llm,
    db,
    prompt=few_shot_prompt,
    verbose=True
)


In [204]:
new_chain("How many white color Levi's shirts I have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirts I have?
SQLQuery:

KeyError: 'Answer'

In [205]:
new_chain("If we sell all Nike t-shirts today, how much revenue we will generate?")



> Entering new SQLDatabaseChain chain...
If we sell all Nike t-shirts today, how much revenue we will generate?
SQLQuery:

KeyError: 'Answer'